### SMS SPAM

In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
sms = pd.read_csv('./\smsspamcollection/SMSSpamCollection.txt',delimiter='\t', header=None, names=['y','x'])

In [3]:
sms.head()

,y,x
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
print sms.shape
print sms.isnull().all()

(5572, 2)
y    False
x    False
dtype: bool


Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [5]:
X = sms.x
Y = sms.y

In [6]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Y = le.fit_transform(Y)
print Y.shape


(5572L,)


Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn import metrics

<p>Convert a collection of text documents to a matrix of token counts </p>
http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [8]:
tf_vectorizer = CountVectorizer()
tf = tf_vectorizer.fit_transform(X)

In [9]:
tf.shape

(5572, 8713)

5 Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [10]:
score1 = cross_val_score(LogisticRegression(random_state=2),tf,y = Y,cv=10,scoring="f1").mean()
print score1

0.932640298361


In [11]:
def write_answer_to_file(answer, filename,rud):
    with open(filename, 'w') as f_out:
        f_out.write(str(round(answer, rud)))

In [12]:
write_answer_to_file(score1,'1.txt',1)

А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:
<ul>
<li>FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB</li>
<li>FreeMsg: Txt: claim your reward of 3 hours talk time</li>
<li>Have you visited the last lecture on physics?</li>
<li>Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99\$</li>
<li>Only 99$</li>
</ul>
Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [13]:
log_regressor = LogisticRegression(random_state = 2)
log_regressor.fit(tf,Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
clf_pipeline = Pipeline(
            [("vectorizer", CountVectorizer()),
            ("classifier", LogisticRegression(random_state = 2))]
        )
clf_pipeline.fit(sms.x, Y)
print clf_pipeline

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
    ...nalty='l2', random_state=2, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])


In [15]:
data = [
    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
    "FreeMsg: Txt: claim your reward of 3 hours talk time",
    "Have you visited the last lecture on physics?",
    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
    "Only 99$"
]

In [16]:
print clf_pipeline.predict(data)

[1 1 0 0 0]


In [17]:
def write_answer_string_to_file(answer, filename):
    with open(filename, 'w') as f_out:
        f_out.write(answer)

In [18]:
write_answer_string_to_file('1 1 0 0 0','2.txt')

7 Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [19]:
out = [];
for i in [(2,2),(3,3),(1,3)]:
    clf_pipeline = Pipeline(
                [("vectorizer", CountVectorizer(ngram_range=i)),
                ("classifier", LogisticRegression(random_state = 2))]
               )
    out.append(cross_val_score(clf_pipeline, sms.x,Y, scoring='f1').mean())


In [20]:
out

[0.77762544198586492, 0.66658748687721592, 0.91066789424772043]

In [21]:
write_answer_string_to_file('0.78 0.67 0.91','3.txt')

8 Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

In [22]:
from sklearn.naive_bayes import MultinomialNB

In [30]:
out = [];
for i in [(2,2),(3,3),(1,3)]:
    clf_pipeline = Pipeline(
                [("vectorizer", CountVectorizer(ngram_range=i)),
                ("classifier", MultinomialNB())]
               )
    out.append(cross_val_score(clf_pipeline, sms.x,Y, scoring='f1').mean())

In [31]:
out

[0.92283823491763339, 0.8380190386913372, 0.94137516326142945]

In [32]:
write_answer_string_to_file('0.92 0.84 0.94','4.txt')

9 Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [26]:
out = [];
for i in [(2,2),(3,3),(1,3)]:
    clf_pipeline = Pipeline(
                [("vectorizer", TfidfVectorizer(ngram_range=i)),
                ("classifier", LogisticRegression(random_state = 2))]
               )
    out.append(cross_val_score(clf_pipeline, sms.x,Y, scoring='f1').mean())

In [27]:
out

[0.33473719450954137, 0.14366231296984541, 0.75395989974937339]

In [28]:
write_answer_string_to_file('-1','5.txt')

#### 8 вопрос

In [37]:
out = [];
for i in [(2,2),(3,3),(1,3)]:
    tf_vectorizer = CountVectorizer(ngram_range=i)
    tf = tf_vectorizer.fit_transform(X)
    a = cross_val_score(MultinomialNB(),tf,y = Y,scoring="f1",cv=10).mean()
    print a
    out.append(a)
    


0.645501517799
0.378719485246
0.888485965606


In [38]:
write_answer_string_to_file('0.65 0.38 0.89','4.txt')